# Sweetness Prediction using lasso and ridge and SVM

In [27]:
# loading libraries 
library(glmnet)
library(caTools)
set.seed(100)

In [28]:
# Reading data
sp <- read.csv("C:/Users/Shubham Shukla/Desktop/internship/new/sweetness_prediction/new_data/ecfp_new.csv")

In [29]:
dim(sp)

[1]  310 2049

In [30]:
# Splitting data into training and testing set

split <- sample.split(sp$logSw, SplitRatio = 0.75)
train <- subset(sp, split == T)
test <- subset(sp, split == F)

In [31]:
train <- subset(train, select = -c(smiles))

In [32]:
test <- subset(test, select = -c(smiles))

In [33]:
testx <- model.matrix(logSw ~. -1, data = test)
testy <- test$logSw

In [34]:
# Converting into matrix 

x <- model.matrix(logSw ~. -1, data = train)
y <- train$logSw

### Using ridge

In [35]:
fit.ridge <- glmnet(x,y, alpha = 0)
# plot(fit.ridge, xvar = "lambda", label = T)

In [36]:
# CV for ridge 

cv.ridge <- cv.glmnet(x,y, alpha = 0)
# plot(cv.ridge)

### Using lasso

In [37]:
cv.lasso <- cv.glmnet(x,y, alpha = 1, nlambda = 100)
# plot(cv.lasso)

In [38]:
# Model fitting using lasso 

fit <- glmnet(x,y, alpha = 1, lambda = cv.lasso$lambda.1se)

In [39]:
fit


Call:  glmnet(x = x, y = y, alpha = 1, lambda = cv.lasso$lambda.1se) 

     Df   %Dev  Lambda
[1,] 81 0.8187 0.06106

In [40]:
model.predict <- predict(fit,testx)
nrow(model.predict)

[1] 78

In [41]:
sse <- sum((model.predict - testy)^2)
sst <- sum((testy - mean(testy))^2)

r_sq <- 1 - (sse/sst)

In [42]:
r_sq

[1] 0.6439931

In [57]:
#Comparison of values in model

df <- data.frame(testy, model.predict)

In [55]:
dim(df)

[1] 78  2

# Using SVM for dataset 

In [63]:
#Library for SVM
library(e1071)

In [67]:
#Modelling using svm

modelsvm <- svm(logSw ~., data = train)
summary(modelsvm)

Warning message in svm.default(x, y, scale = scale, ..., na.action = na.action):
"Variable(s) 'X.7' and 'X.8' and 'X.13' and 'X.15' and 'X.16' and 'X.17' and 'X.19' and 'X.22' and 'X.23' and 'X.34' and 'X.38' and 'X.39' and 'X.42' and 'X.44' and 'X.45' and 'X.47' and 'X.49' and 'X.53' and 'X.55' and 'X.56' and 'X.57' and 'X.59' and 'X.60' and 'X.64' and 'X.65' and 'X.68' and 'X.69' and 'X.70' and 'X.71' and 'X.72' and 'X.74' and 'X.76' and 'X.77' and 'X.78' and 'X.81' and 'X.82' and 'X.83' and 'X.91' and 'X.94' and 'X.96' and 'X.98' and 'X.100' and 'X.101' and 'X.103' and 'X.106' and 'X.111' and 'X.113' and 'X.114' and 'X.115' and 'X.116' and 'X.121' and 'X.126' and 'X.129' and 'X.131' and 'X.134' and 'X.135' and 'X.137' and 'X.140' and 'X.141' and 'X.144' and 'X.146' and 'X.150' and 'X.151' and 'X.155' and 'X.163' and 'X.166' and 'X.167' and 'X.168' and 'X.169' and 'X.171' and 'X.173' and 'X.180' and 'X.182' and 'X.183' and 'X.185' and 'X.186' and 'X.190' and 'X.191' and 'X.192' and '


Call:
svm(formula = logSw ~ ., data = train)


Parameters:
   SVM-Type:  eps-regression 
 SVM-Kernel:  radial 
       cost:  1 
      gamma:  0.0004885198 
    epsilon:  0.1 


Number of Support Vectors:  201






In [70]:
pred_svm <- predict(modelsvm, newdata = test)

In [92]:
sse <- sum((test$logSw - pred_svm)^2)
sst <- sum((test$logSw - mean(test$logSw))^2)

1 - sse/sst

[1] 0.2648624

In [97]:
errors <- test$logSw - pred_svm

In [99]:
data.frame(test$logSw, pred_svm, errors, (pred_svm+errors))

,test.logSw,pred_svm,errors,X.pred_svm...errors.
7,5.88,3.651355,2.228645178,5.88
12,3.60,3.555252,0.044747891,3.60
15,4.20,3.715420,0.484580329,4.20
24,5.00,3.693304,1.306695614,5.00
27,4.40,3.976444,0.423556299,4.40
28,1.74,3.211901,-1.471901068,1.74
32,4.00,3.988235,0.011765421,4.00
34,3.70,3.942297,-0.242296676,3.70
36,3.48,3.484779,-0.004779355,3.48
39,4.11,3.503663,0.606336946,4.11
